In [41]:
# Cell 1: Imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import math
import copy

In [42]:
# Cell 2: Create directories
os.makedirs('result_weight_rejection/plots', exist_ok=True)
os.makedirs('result_weight_rejection/plots/individual_clients', exist_ok=True)

In [43]:
# Cell 3: Configuration
# ============================================
# FEDERATED LEARNING PARAMETERS
# ============================================
NUM_CLIENTS = 5      # 5-split data
NUM_ROUNDS = 10       # Number of communication rounds
BATCH_SIZE = 64

# WEIGHT REJECTION PARAMETERS
CHECK_INTERVAL = 5    # Check accuracy every 5 epochs
MAX_EPOCHS = 100      # Maximum epochs per round

print("=" * 60)
print("FEDERATED LEARNING WITH WEIGHT REJECTION")
print("=" * 60)
print(f"Number of Clients: {NUM_CLIENTS}")
print(f"Communication Rounds: {NUM_ROUNDS}")
print(f"Check Interval: Every {CHECK_INTERVAL} epochs")
print(f"Max Epochs per Round: {MAX_EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print("=" * 60 + "\n")

FEDERATED LEARNING WITH WEIGHT REJECTION
Number of Clients: 5
Communication Rounds: 10
Check Interval: Every 5 epochs
Max Epochs per Round: 100
Batch Size: 64



In [44]:
# Cell 4: Load and normalize client data
# ============================================
# LOAD AND NORMALIZE CLIENT DATA
# ============================================
print("=" * 60)
print("LOADING AND NORMALIZING CLIENT DATA")
print("=" * 60)

client_data = []
for i in range(1, NUM_CLIENTS + 1):
    data_path = f'data/5_split_data/mnist_part{i}.npz'
    
    # Load .npz file
    data = np.load(data_path)
    
    # Get the data
    x_train = data['x_train']
    y_train = data['y_train']
    x_test = data['x_test']
    y_test = data['y_test']
    
    # Normalize
    x_train = x_train / 255.0
    x_test = x_test / 255.0
    
    # Flatten
    x_train_flatten = x_train.reshape(len(x_train), 28*28)
    x_test_flatten = x_test.reshape(len(x_test), 28*28)
    
    # Store normalized and flattened data
    normalized_data = {
        'x_train': x_train_flatten,
        'y_train': y_train,
        'x_test': x_test_flatten,
        'y_test': y_test
    }
    
    client_data.append(normalized_data)
    print(f"✓ Client {i} - Train: {len(x_train_flatten)}, Test: {len(x_test_flatten)}")

print(f"\n✓ Total clients loaded: {len(client_data)}\n")

LOADING AND NORMALIZING CLIENT DATA
✓ Client 1 - Train: 11999, Test: 2001
✓ Client 2 - Train: 11999, Test: 2001
✓ Client 3 - Train: 11999, Test: 2001
✓ Client 4 - Train: 11999, Test: 2001
✓ Client 5 - Train: 11999, Test: 2001

✓ Total clients loaded: 5



In [45]:
# Cell 5: Define model architecture
# ============================================
# DEFINE MODEL ARCHITECTURE
# ============================================
def create_model():
    """Model architecture"""
    model = keras.Sequential([
        # First hidden layer - Feature extraction
        keras.layers.Dense(128, input_shape=(784,), activation="relu"),
        keras.layers.BatchNormalization(), 
        keras.layers.Dropout(0.3),
        
        # Second hidden layer - Pattern recognition
        keras.layers.Dense(64, activation="relu"),
        keras.layers.BatchNormalization(), 
        keras.layers.Dropout(0.2), 
        
        # Third hidden layer - Feature refinement
        keras.layers.Dense(32, activation="relu"),
        keras.layers.BatchNormalization(), 
        keras.layers.Dropout(0.2), 
        
        # Output layer - Classification
        keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0005),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [46]:
# Cell 6: Federated averaging function
# ============================================
# FEDERATED AVERAGING FUNCTION
# ============================================
def average_weights(weights_list):
    """Average weights from all clients"""
    averaged_weights = []
    for weights_tuple in zip(*weights_list):
        averaged_weights.append(np.mean(weights_tuple, axis=0))
    return averaged_weights

In [47]:
# Cell 7: Initialize global model and tracking variables
# ============================================
# INITIALIZE GLOBAL MODEL
# ============================================
print("Initializing global model...")
global_model = create_model()
global_model.summary()
global_weights = global_model.get_weights()
print("\n✓ Global model initialized\n")

# Track accuracy for each client
client_test_accuracies = {f'client_{i}': [] for i in range(NUM_CLIENTS)}

# Track previous accuracy for each client (start with 0)
client_prev_accuracy = {f'client_{i}': 0.0 for i in range(NUM_CLIENTS)}

# Track epochs used per client per round
client_epochs_used = {f'client_{i}': [] for i in range(NUM_CLIENTS)}

# Track weight rejections per client per round
client_weight_rejections = {f'client_{i}': [] for i in range(NUM_CLIENTS)}

# Track each client's best weights (NOT using global averaged weights)
client_best_weights = {f'client_{i}': None for i in range(NUM_CLIENTS)}

all_training_histories = []
# Store training histories

Initializing global model...


Model: "sequential_56"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_224 (Dense)               │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_168         │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_168 (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_225 (Dense)               │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_169         │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_169 (Dropout)           │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_226 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_170         │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_170 (Dropout)           │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_227 (Dense)               │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 112,042 (437.66 KB)

 Trainable params: 111,594 (435.91 KB)

 Non-trainable params: 448 (1.75 KB)


✓ Global model initialized



In [48]:
# Cell 8: Federated learning training loop with WEIGHT REJECTION
# ============================================
# FEDERATED LEARNING WITH WEIGHT REJECTION
# ============================================
print("=" * 60)
print("STARTING FEDERATED LEARNING (WEIGHT REJECTION)")
print("=" * 60 + "\n")

for round_num in range(NUM_ROUNDS):
    print(f"\n{'='*70}")
    print(f"COMMUNICATION ROUND {round_num + 1}/{NUM_ROUNDS}")
    print(f"{'='*70}\n")
    
    local_weights = []
    round_histories = []
    
    # Train each client
    for client_id in range(NUM_CLIENTS):
        print(f"\n{'─'*70}")
        print(f"[Client {client_id + 1}] Starting training with weight rejection...")
        print(f"[Client {client_id + 1}] Previous best accuracy: {client_prev_accuracy[f'client_{client_id}']*100:.2f}%")
        print(f"{'─'*70}")
        
        # Create fresh model
        client_model = create_model()
        
        # Load client's own best weights (or global weights if first round)
        if client_best_weights[f'client_{client_id}'] is not None:
            client_model.set_weights(client_best_weights[f'client_{client_id}'])
            print(f"   → Using client's own best weights from previous round")
        else:
            client_model.set_weights(global_weights)
            print(f"   → Using initial global weights (first round)")
        
        # Get client's data
        x_train = client_data[client_id]['x_train']
        y_train = client_data[client_id]['y_train']
        x_test = client_data[client_id]['x_test']
        y_test = client_data[client_id]['y_test']
        
        # Track best weights and accuracy
        best_weights = copy.deepcopy(client_model.get_weights())
        best_accuracy = client_prev_accuracy[f'client_{client_id}']
        
        training_history = {'accuracy': [], 'val_accuracy': [], 'loss': [], 'val_loss': []}
        total_epochs = 0
        rejections = 0
        
        # For first round, train only 5 epochs
        if round_num == 0:
            print(f"   Training 5 epochs (Round 1 - baseline)...")
            
            history = client_model.fit(
                x_train, y_train,
                epochs=CHECK_INTERVAL,
                batch_size=BATCH_SIZE,
                validation_split=0.1,
                verbose=0
            )
            
            total_epochs = CHECK_INTERVAL
            
            # Store history
            training_history['accuracy'].extend(history.history['accuracy'])
            training_history['val_accuracy'].extend(history.history['val_accuracy'])
            training_history['loss'].extend(history.history['loss'])
            training_history['val_loss'].extend(history.history['val_loss'])
            
            # Evaluate on test set
            _, current_accuracy = client_model.evaluate(x_test, y_test, verbose=0)
            
            # Accept weights (first round baseline)
            best_weights = copy.deepcopy(client_model.get_weights())
            best_accuracy = current_accuracy
            client_best_weights[f'client_{client_id}'] = copy.deepcopy(best_weights)
            print(f"   Epoch {total_epochs}: Test Acc = {current_accuracy*100:.2f}% ✓ BASELINE SET")
        
        else:
            # For subsequent rounds, train in intervals and check for improvement
            weight_accepted = False
            
            for epoch_checkpoint in range(CHECK_INTERVAL, MAX_EPOCHS + 1, CHECK_INTERVAL):
                # Calculate how many epochs to train
                epochs_to_train = CHECK_INTERVAL
                
                print(f"   Training epochs {total_epochs + 1} to {total_epochs + epochs_to_train}...")
                
                # Train for CHECK_INTERVAL epochs
                history = client_model.fit(
                    x_train, y_train,
                    epochs=epochs_to_train,
                    batch_size=BATCH_SIZE,
                    validation_split=0.1,
                    verbose=0
                )
                
                total_epochs += epochs_to_train
                
                # Store history
                training_history['accuracy'].extend(history.history['accuracy'])
                training_history['val_accuracy'].extend(history.history['val_accuracy'])
                training_history['loss'].extend(history.history['loss'])
                training_history['val_loss'].extend(history.history['val_loss'])
                
                # Evaluate on test set
                _, current_accuracy = client_model.evaluate(x_test, y_test, verbose=0)
                
                print(f"   Epoch {total_epochs}: Test Acc = {current_accuracy*100:.2f}%", end="")
                
                # Check if current accuracy is better than previous round's best
                if current_accuracy > best_accuracy:
                    # Accept weights and move to next client
                    best_weights = copy.deepcopy(client_model.get_weights())
                    client_best_weights[f'client_{client_id}'] = copy.deepcopy(best_weights)
                    improvement = (current_accuracy - client_prev_accuracy[f'client_{client_id}']) * 100
                    print(f" ✓ ACCEPTED (improved by {improvement:+.2f}%) - Moving to next client")
                    weight_accepted = True
                    break
                else:
                    # Reject weights and continue training
                    rejections += 1
                    print(f" ✗ REJECTED (worse than {best_accuracy*100:.2f}%)")
                
                # If reached max epochs without improvement
                if total_epochs >= MAX_EPOCHS:
                    print(f"   ⚠ Reached max epochs ({MAX_EPOCHS}) - Using previous best weights")
                    client_model.set_weights(best_weights)
                    break
            
            # If no improvement found after all attempts
            if not weight_accepted:
                print(f"   → No improvement found. Keeping previous round's weights.")
                client_model.set_weights(best_weights)
                # Ensure client keeps their previous best weights
                if client_best_weights[f'client_{client_id}'] is None:
                    client_best_weights[f'client_{client_id}'] = copy.deepcopy(best_weights)
        
        # Create history object
        class HistoryObject:
            def __init__(self, history_dict):
                self.history = history_dict
        
        history_obj = HistoryObject(training_history)
        
        # Store history
        round_histories.append({
            'client_id': client_id,
            'round': round_num,
            'history': history_obj,
            'epochs_used': total_epochs
        })
        
        # Final evaluation with best weights
        client_model.set_weights(best_weights)
        loss, final_accuracy = client_model.evaluate(x_test, y_test, verbose=0)
        
        # Update tracking
        client_prev_accuracy[f'client_{client_id}'] = final_accuracy
        client_test_accuracies[f'client_{client_id}'].append(final_accuracy)
        client_epochs_used[f'client_{client_id}'].append(total_epochs)
        client_weight_rejections[f'client_{client_id}'].append(rejections)
        
        print(f"\n{'─'*70}")
        print(f"🎯 Client {client_id + 1} Results:")
        print(f"   - Final Accuracy: {final_accuracy * 100:.2f}%")
        print(f"   - Epochs Used: {total_epochs}")
        print(f"   - Weight Rejections: {rejections}")
        print(f"   - Improvement from Round Start: {(final_accuracy - client_prev_accuracy[f'client_{client_id}']) * 100:+.2f}%")
        print(f"{'─'*70}\n")
        
        # Store best weights for averaging
        local_weights.append(best_weights)
    
    # Store histories
    all_training_histories.append(round_histories)
    
    # Federated Averaging
    print(f"\n{'🔄'*35}")
    print("WEIGHT EXCHANGE - FEDERATED AVERAGING")
    print(f"{'🔄'*35}")
    global_weights = average_weights(local_weights)
    global_model.set_weights(global_weights)
    print("✓ Global weights updated")
    
    # Print round summary
    round_epochs = [client_epochs_used[f'client_{i}'][-1] for i in range(NUM_CLIENTS)]
    round_accs = [client_test_accuracies[f'client_{i}'][-1] * 100 for i in range(NUM_CLIENTS)]
    round_rejections = [client_weight_rejections[f'client_{i}'][-1] for i in range(NUM_CLIENTS)]
    
    print(f"\n📊 Round {round_num + 1} Summary:")
    print(f"   - Avg Accuracy: {np.mean(round_accs):.2f}%")
    print(f"   - Min/Max Accuracy: {np.min(round_accs):.2f}% / {np.max(round_accs):.2f}%")
    print(f"   - Total Weight Rejections: {sum(round_rejections)}")
    print(f"{'🔄'*35}\n")

print("\n" + "="*70)
print("FEDERATED LEARNING COMPLETE!")
print("="*70 + "\n")


STARTING FEDERATED LEARNING (WEIGHT REJECTION)


COMMUNICATION ROUND 1/10


──────────────────────────────────────────────────────────────────────
[Client 1] Starting training with weight rejection...
[Client 1] Previous best accuracy: 0.00%
──────────────────────────────────────────────────────────────────────
   → Using initial global weights (first round)
   Training 5 epochs (Round 1 - baseline)...
   Epoch 5: Test Acc = 89.16% ✓ BASELINE SET

──────────────────────────────────────────────────────────────────────
🎯 Client 1 Results:
   - Final Accuracy: 89.16%
   - Epochs Used: 5
   - Weight Rejections: 0
   - Improvement from Round Start: +0.00%
──────────────────────────────────────────────────────────────────────


──────────────────────────────────────────────────────────────────────
[Client 2] Starting training with weight rejection...
[Client 2] Previous best accuracy: 0.00%
──────────────────────────────────────────────────────────────────────
   → Using initial global weigh

In [49]:
# Cell 9: Display final results
# ============================================
# DISPLAY FINAL RESULTS
# ============================================
print("FINAL RESULTS:")
print("-" * 90)
print(f"{'Client':<10} {'Final Acc':<15} {'Total Epochs':<15} {'Total Rejections':<20}")
print("-" * 90)

for client_id in range(NUM_CLIENTS):
    final_acc = client_test_accuracies[f'client_{client_id}'][-1]
    total_epochs = sum(client_epochs_used[f'client_{client_id}'])
    total_rejections = sum(client_weight_rejections[f'client_{client_id}'])
    print(f"{client_id + 1:<10} {final_acc*100:.2f}%{'':<8} {total_epochs:<15} {total_rejections:<20}")

print("-" * 90)

# Overall summary
all_final_accs = [client_test_accuracies[f'client_{i}'][-1] * 100 for i in range(NUM_CLIENTS)]
all_total_epochs = [sum(client_epochs_used[f'client_{i}']) for i in range(NUM_CLIENTS)]
all_total_rejections = [sum(client_weight_rejections[f'client_{i}']) for i in range(NUM_CLIENTS)]

print(f"\n📊 OVERALL SUMMARY:")
print(f"   - Average Final Accuracy: {np.mean(all_final_accs):.2f}%")
print(f"   - Average Total Epochs: {np.mean(all_total_epochs):.1f}")
print(f"   - Total Training Epochs (all clients): {sum(all_total_epochs)}")
print(f"   - Total Weight Rejections: {sum(all_total_rejections)}")
print("-" * 90 + "\n")

FINAL RESULTS:
------------------------------------------------------------------------------------------
Client     Final Acc       Total Epochs    Total Rejections    
------------------------------------------------------------------------------------------
1          95.00%         450             82                  
2          95.05%         400             73                  
3          95.95%         445             80                  
4          95.80%         390             69                  
5          96.05%         500             93                  
------------------------------------------------------------------------------------------

📊 OVERALL SUMMARY:
   - Average Final Accuracy: 95.57%
   - Average Total Epochs: 437.0
   - Total Training Epochs (all clients): 2185
   - Total Weight Rejections: 397
------------------------------------------------------------------------------------------



In [50]:
# Cell 10: Generate plots
# ============================================
# GENERATE PLOTS
# ============================================
print("=" * 70)
print("GENERATING PLOTS")
print("=" * 70)

# ────────────────────────────────────────────
# Plot 1: Individual client accuracy plots
# ────────────────────────────────────────────
print("\n1. Creating individual client test accuracy plots...")

for client_id in range(NUM_CLIENTS):
    plt.figure(figsize=(10, 6))
    
    accuracies = [acc * 100 for acc in client_test_accuracies[f'client_{client_id}']]
    
    plt.plot(range(1, NUM_ROUNDS + 1), accuracies, 
             marker='o', linewidth=3, markersize=10, 
             color='green', label=f'Client {client_id + 1}')
    
    # Add accuracy values on points
    for i, acc in enumerate(accuracies):
        plt.text(i + 1, acc + 0.3, f'{acc:.2f}%', 
                ha='center', fontsize=10, fontweight='bold')
    
    plt.title(f'Client {client_id + 1}: Test Accuracy vs Communication Rounds (Weight Rejection)', 
              fontsize=16, fontweight='bold')
    plt.xlabel('Communication Round', fontsize=13)
    plt.ylabel('Test Accuracy (%)', fontsize=13)
    plt.legend(loc='lower right', fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.xticks(range(1, NUM_ROUNDS + 1))
    plt.ylim([85, 100])
    
    save_path = f'result_weight_rejection/plots/individual_clients/client_{client_id + 1}_accuracy.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"   ✓ Saved: client_{client_id + 1}_accuracy.png")
    plt.close()

# ────────────────────────────────────────────
# Plot 2: All clients overlay
# ────────────────────────────────────────────
print("\n2. Creating overlay plot of all clients...")

plt.figure(figsize=(12, 7))
colors = plt.cm.tab10(np.linspace(0, 1, NUM_CLIENTS))

for client_id in range(NUM_CLIENTS):
    accuracies = [acc * 100 for acc in client_test_accuracies[f'client_{client_id}']]
    plt.plot(range(1, NUM_ROUNDS + 1), accuracies, 
             marker='o', linewidth=2.5, markersize=8, 
             color=colors[client_id],
             label=f'Client {client_id + 1}')

plt.title('Test Accuracy vs Communication Rounds (All Clients - Weight Rejection)', 
          fontsize=16, fontweight='bold')
plt.xlabel('Communication Round', fontsize=13)
plt.ylabel('Test Accuracy (%)', fontsize=13)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, NUM_ROUNDS + 1))
plt.ylim([85, 100])

save_path = 'result_weight_rejection/plots/all_clients_accuracy_overlay.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: all_clients_accuracy_overlay.png")
plt.close()

# ────────────────────────────────────────────
# Plot 3: Weight rejections heatmap
# ────────────────────────────────────────────
print("\n3. Creating weight rejections heatmap...")

plt.figure(figsize=(10, 6))

rejections_matrix = np.zeros((NUM_CLIENTS, NUM_ROUNDS))
for client_id in range(NUM_CLIENTS):
    for round_num in range(NUM_ROUNDS):
        rejections_matrix[client_id, round_num] = client_weight_rejections[f'client_{client_id}'][round_num]

plt.imshow(rejections_matrix, cmap='Reds', aspect='auto')
plt.colorbar(label='Weight Rejections')

for i in range(NUM_CLIENTS):
    for j in range(NUM_ROUNDS):
        plt.text(j, i, f'{int(rejections_matrix[i, j])}', 
                ha='center', va='center', fontsize=12, fontweight='bold')

plt.title('Weight Rejections per Client per Round', fontsize=16, fontweight='bold')
plt.xlabel('Communication Round', fontsize=13)
plt.ylabel('Client', fontsize=13)
plt.xticks(range(NUM_ROUNDS), range(1, NUM_ROUNDS + 1))
plt.yticks(range(NUM_CLIENTS), range(1, NUM_CLIENTS + 1))

save_path = 'result_weight_rejection/plots/weight_rejections_heatmap.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: weight_rejections_heatmap.png")
plt.close()

# ────────────────────────────────────────────
# Plot 4: Average accuracy with std
# ────────────────────────────────────────────
print("\n4. Creating average accuracy plot...")

plt.figure(figsize=(12, 7))

avg_accuracies = []
std_accuracies = []
for round_num in range(NUM_ROUNDS):
    round_accs = [client_test_accuracies[f'client_{i}'][round_num] * 100 for i in range(NUM_CLIENTS)]
    avg_accuracies.append(np.mean(round_accs))
    std_accuracies.append(np.std(round_accs))

plt.plot(range(1, NUM_ROUNDS + 1), avg_accuracies, 
         marker='s', linewidth=3, markersize=10, 
         color='purple', label=f'Average ({NUM_CLIENTS} clients)')

plt.fill_between(range(1, NUM_ROUNDS + 1), 
                 np.array(avg_accuracies) - np.array(std_accuracies),
                 np.array(avg_accuracies) + np.array(std_accuracies),
                 alpha=0.2, color='purple')

for i, acc in enumerate(avg_accuracies):
    plt.text(i + 1, acc + 0.5, f'{acc:.2f}%', 
            ha='center', fontsize=10, fontweight='bold')

plt.title('Average Test Accuracy vs Communication Rounds (Weight Rejection)', 
          fontsize=16, fontweight='bold')
plt.xlabel('Communication Round', fontsize=13)
plt.ylabel('Test Accuracy (%)', fontsize=13)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, NUM_ROUNDS + 1))
plt.ylim([85, 100])

save_path = 'result_weight_rejection/plots/average_accuracy.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: average_accuracy.png")
plt.close()

print("\n" + "="*70)
print("ALL PLOTS GENERATED!")
print("="*70)

GENERATING PLOTS

1. Creating individual client test accuracy plots...
   ✓ Saved: client_1_accuracy.png
   ✓ Saved: client_2_accuracy.png
   ✓ Saved: client_3_accuracy.png
   ✓ Saved: client_4_accuracy.png
   ✓ Saved: client_5_accuracy.png

2. Creating overlay plot of all clients...
   ✓ Saved: all_clients_accuracy_overlay.png

3. Creating weight rejections heatmap...
   ✓ Saved: weight_rejections_heatmap.png

4. Creating average accuracy plot...
   ✓ Saved: average_accuracy.png

ALL PLOTS GENERATED!


In [51]:
# Cell 11: Save results
# ============================================
# SAVE RESULTS
# ============================================
os.makedirs('result_weight_rejection', exist_ok=True)

# Save results
results_data = {
    'client_test_accuracies': client_test_accuracies,
    'client_epochs_used': client_epochs_used,
    'client_weight_rejections': client_weight_rejections,
    'num_clients': NUM_CLIENTS,
    'num_rounds': NUM_ROUNDS,
    'check_interval': CHECK_INTERVAL,
    'max_epochs': MAX_EPOCHS
}
np.save('result_weight_rejection/training_results.npy', results_data)
print(f"✓ Results saved to: result_weight_rejection/training_results.npy")

# Save global model
model_save_path = 'result_weight_rejection/federated_global_model.h5'
global_model.save(model_save_path)
print(f"✓ Final global model saved to: {model_save_path}")

print("\n✅ FEDERATED LEARNING WITH WEIGHT REJECTION COMPLETE!")
print(f"   Total Epochs Used: {sum(all_total_epochs)}")
print(f"   Total Weight Rejections: {sum(all_total_rejections)}")
print(f"   Average Final Accuracy: {np.mean(all_final_accs):.2f}%")

✓ Results saved to: result_weight_rejection/training_results.npy
✓ Final global model saved to: result_weight_rejection/federated_global_model.h5

✅ FEDERATED LEARNING WITH WEIGHT REJECTION COMPLETE!
   Total Epochs Used: 2185
   Total Weight Rejections: 397
   Average Final Accuracy: 95.57%
